# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [1]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from sidecar import Sidecar
from ipywidgets import VBox
from itertools import product

### First generate some data.

In [2]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [3]:
# Create a plot instance
plot_l = fpl.Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the `1:1` button to toggle this.

In [4]:
plot_l.camera.maintain_aspect = False

### reset the plot area

In [5]:
plot_l.auto_scale(maintain_aspect=True)

### Graphic features support slicing! :D

In [6]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"

In [6]:
cosine_graphic.colors[90:] = "red"

In [6]:
cosine_graphic.colors[60] = "w"

In [6]:
# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

In [6]:
# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [7]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [8]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

FeatureEvent @ 0x7f820c410bd0
type: colors
pick_info: {'index': range(15, 50, 3), 'collection-index': None, 'world_object': <weakproxy at 0x7f82418127f0 to Line at 0x7f824187cc10>, 'new_data': array([[0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]], dtype=float32)}



### Graphic data is itself also indexable

In [9]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]

In [9]:
cosine_graphic.data[90:, 1] = 7

In [9]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:31: UserWarning: converting int64 array to int32
  warn(f"converting {array.dtype} array to int32")


### Toggle the presence of a graphic within the scene

In [10]:
sinc_graphic.present = False

In [11]:
sinc_graphic.present = True

### You can create callbacks to this too, for example to re-scale the plot w.r.t. graphics that are present in the scene


In [12]:
sinc_graphic.present.add_event_handler(plot_l.auto_scale)

sinc_graphic.present = False

In [13]:
sinc_graphic.present = True

In [15]:
plot_l.close()
sc.close()

### Can also have fancy indexing of colormaps

In [16]:
plot = fpl.Plot()

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

RFBOutputContext()

In [17]:
plot["sine"].cmap = "jet"

In [18]:
plot["sine"].cmap.values = sine[:, 1]

In [19]:
plot["sine"].cmap.values = cosine[:, 1]

In [20]:
plot["sine"].cmap = "viridis"

In [21]:
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

In [22]:
plot.close()
sc.close()

## Line Collections

Generate some data

In [23]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center


spatial_dims = (50, 50)

# this makes 16 circles, so we can create 16 cmap values, so it will use these values to set the
# color of the line based by using the cmap as a LUT with the corresponding cmap_value
circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

pos_xy = np.vstack(circles)

### set cmap values as LUT

In [24]:
plot = fpl.Plot()

# highest values, lowest values, mid-high values, mid values
cmap_values = cmap_values = [
    0, 1, 1, 2,
    0, 0, 1, 1,
    2, 2, 3, 3,
    1, 1, 1, 5
]

plot.add_line_collection(circles, cmap="tab10", cmap_values=cmap_values, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [25]:
plot.close()
sc.close()

### can also set colors

In [26]:
plot = fpl.Plot()

colors = ["green"] * 4 + ["red"] * 4 + ["yellow"] * 4 + ["w"] * 4

plot.add_line_collection(circles, colors=colors, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [27]:
plot.close()
sc.close()

## Line Stack

Generate some data

In [28]:
xs = np.linspace(0, 100, 1000)
# sine wave
ys = np.sin(xs) * 20

# make 25 lines
data = np.vstack([ys] * 25)

In [29]:
plot = fpl.Plot()

# line stack takes all the same arguments as line collection and behaves similarly
plot.add_line_stack(data, cmap="jet")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show(maintain_aspect=False))

RFBOutputContext()

In [30]:
plot.close()
sc.close()

## Linear Selector

Creates a horizontal or vertical line slider. Can be synced to an `ipywidget IntSlider`. Like other `Graphic` types, it can be linked to events. 

In [31]:
plot = fpl.Plot()

# data to plot
xs = np.linspace(0, 100, 1000)
sine = np.sin(xs) * 20

# make sine along x axis
sine_graphic = plot.add_line(np.column_stack([xs, sine]).astype(np.float32))

# make some selectors
selector = sine_graphic.add_linear_selector()
selector2 = sine_graphic.add_linear_selector(20)
selector3 = sine_graphic.add_linear_selector(40)

# add ability to synchronize selectors
ss = Synchronizer(selector, selector2, selector3)

plot.auto_scale()

# fastplotlib LineSelector can make an ipywidget slider and return it :D 
ipywidget_slider = selector.make_ipywidget_slider()

sc = Sidecar(title="linear selectors")

with sc:
    display(VBox([plot.show(), ipywidget_slider]))

RFBOutputContext()

### can add an event handler to a selector

In [32]:
def set_color_at_index(ev):
    # changes the color at the index where the slider is
    ix = ev.pick_info["selected_index"]
    g = ev.pick_info["graphic"].parent
    g.colors[ix] = "magenta"

selector.selection.add_event_handler(set_color_at_index)

In [33]:
plot.close()
sc.close()

## Linear Region Selector

Creates a linear region bounded graphic which can be moved along either the x-axis or y-axis.
Allows sub-selecting data from a `Graphic` or from multiple `Graphic` objects.

In [34]:
# data to plot
xs = np.linspace(0, 250, 500)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.Plot()

# sines and cosines
sines = [sine] * 5
cosines = [cosine] * 5

# make line stack
line_stack = plot.add_line_stack(sines + cosines, separation=50)

# make selector
stack_selector = line_stack.add_linear_region_selector()

def update_color(ev):
    selected_indices = stack_selector.get_selected_indices()
    
    for i in range(len(selected_indices)):
        line_stack.graphics[i].colors = "w"
        line_stack.graphics[i].colors[selected_indices[i]] = "magenta"

stack_selector.selection.add_event_handler(update_color)

sc = Sidecar(title="linear region selector")

with sc: 
    display(plot.show())

RFBOutputContext()

In [35]:
plot.close()
sc.close()

## Heatmap

In [37]:
temporal = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/temporal.npy')

In [38]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "traces"]])

# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

# add linear selector to heatmap
selector = heatmap.add_linear_region_selector()

sc = Sidecar(title="heatmap interactive")

with sc:
    display(plot.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### plot interaction using event handlers

In [40]:
# define event handler
def color_change(ev):
    selected_indices = ev.pick_info["selected_indices"]
    
    for i in range(len(temporal_stack.graphics)):
        temporal_stack.graphics[i].colors = "magenta"
        temporal_stack.graphics[i].colors[selected_indices] = "white"

# add event handler
selector.selection.add_event_handler(color_change)

In [41]:
plot.close()
sc.close()